In [1]:
import pandas as pd

df_cust = pd.read_csv('../data/raw/csv/customers.csv')
df_pro = pd.read_csv('../data/raw/csv/products.csv')
df_ord = pd.read_csv('../data/raw/csv/orders.csv')
df_ordit = pd.read_csv('../data/raw/csv/order_items.csv')

In [2]:
# converting the datetime columns from string to datetime
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_cols:
    df_ord[col] = pd.to_datetime(df_ord[col], errors='coerce') # the columns are passed as a list and iterated to convert into datetime
print(df_ord[date_cols].dtypes) 
print(df_ord[date_cols].isna().sum())



order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64


In [3]:
# adding a new column delayed_days, which means delivery_date - estimated_delivery_date

df_ord['delayed_days'] = (df_ord['order_delivered_customer_date'] - df_ord['order_estimated_delivery_date']).dt.days
df_ord['delayed_days'].head(10)


# cleaning the orders table

# order_approved_at means the order was cancelled
# order_delivered_carrier_date isnull() means the order never shipped
# order_delivered_customer_date isnull() means the order is never delivered

# adding a new boolean column if the dilverey date is not null than true else False
df_ord['order_completed'] = df_ord['order_delivered_customer_date'].notna()
df_ord['order_completed'].value_counts()

# adding a new boolean column for the carrier date for the same
df_ord['order_shipped'] = df_ord['order_delivered_carrier_date'].notna()
df_ord['order_shipped'].value_counts()

# adding a new boolean column for the order approved for the same
df_ord['order_approved'] = df_ord['order_approved_at'].notna()
df_ord['order_approved'].value_counts()

order_approved
True     99281
False      160
Name: count, dtype: int64

In [4]:
# checking the integrity of the tables
# validating FKs and finding orphans

# finding orphan orders(Orders which does not have any customers)
order_missing_customers = set(df_ord['customer_id']).difference(set(df_cust['customer_id']))
len(order_missing_customers), list(order_missing_customers)[:10]

# finding orphan order_items without order_id
items_missing_orders = set(df_ordit['order_id']).difference(set(df_ord['order_id']))
len(items_missing_orders), list(items_missing_orders)[:10]

# finding oprhan order_items without products
items_missing_products = set(df_ordit['product_id']).difference(set(df_pro['product_id']))
len(items_missing_products), list(items_missing_products)[:10]

# explaination : the columns are converted into set which means it contains only distinct values
#                after that .difference() is like customers which are present in orders table but not in the customers table
#                here the sequence matters, what you write before the difference and after it
#                after the len() function is used to check is there is any missing values, if yes than extra steps are needed
     

(0, [])

In [5]:
# making order_items table ready for calculations

df_ordit['price'] = pd.to_numeric(df_ordit['price'], errors='coerce') # here as the price is already in float64 this will make no 
# difference but in maximum cases it is needed to be checked in real world data


df_ordit['quantity'] = 1

# calc sales
df_ordit['sales'] = df_ordit['price'] * df_ordit['quantity']

df_ordit[['price','quantity','sales']].describe() # just some quick maths
df_ordit['price'].isna().sum() # to check nulls in the price column

np.int64(0)

In [6]:
# calculating aggregations for the orders table 

orders_agg = df_ordit.groupby('order_id').agg(
    order_total = ('sales','sum'), # it is a new column which sums the sales for the same order_id
    total_freight = ('freight_value','sum'), # same as above for the freight
    num_items = ('order_item_id','nunique')
).reset_index() # as the groupby() coverts the targeted column into index so reset_index() converts it back to the column

# df_ord = df_ord.merge(orders_agg, on='order_id', how='left')

# filling all the missing values with zero for better calculations
df_ord['order_total'] = df_ord['order_total'].fillna(0)
df_ord['total_freight'] = df_ord['total_freight'].fillna(0)
df_ord['num_items'] = df_ord['num_items'].fillna(0)

# df_ord = df_ord.drop(columns=[
#     'order_total_x', 'total_freight_x', 'num_items_x',
#     'order_total_y', 'total_freight_y', 'num_items_y'
# ])

df_ord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 8   delayed_days                   96476 non-null  float64       
 9   order_completed                99441 non-null  bool          
 10  order_shipped                  99441 non-null  bool          
 11  order_approved 

In [7]:
# calculating aggregations for customers table

id_map = df_cust.set_index('customer_id')['customer_unique_id'].to_dict() # it creates a dictionary where the key is the customer_id and
# value is the customer_unique_id

df_ord['customer_unique_id'] = df_ord['customer_id'].map(id_map) # the id_map contains the id and unique_id, the customer_id of 
# orders table is matched with the id of the customers table and the unique id for that customer will be stored as the unique_id in 
# the orders table. 

df_ord['customer_unique_id'].head()

0    7c396fd4830fd04220f754e42b4e5bff
1    af07308b275d755c9edb36a90c618231
2    3a653a41f6f9fc3d2a113cf8398680e8
3    7c142cf63193a1473d2e66489a9ae977
4    72632f0f9dd73dfee390c9b22eb56dd6
Name: customer_unique_id, dtype: object

In [ ]:
# aggregating according to the timeline of the orders table for the customers

cutoff = df_ord['order_purchase_timestamp'].max() - pd.Timedelta(days=365)

cust_agg = df_ord.groupby('customer_unique_id').agg(
    first_order_date = ('order_purchase_timestamp','min'),
    last_order_date = ('order_purchase_timestamp','max'),
    num_orders = ('order_id','nunique'),
    total_revenue = ('order_total','sum')
).reset_index()

cust_agg['active'] =  cust_agg['last_order_date'] >= cutoff
cust_agg['active'] = cust_agg['active'].fillna(False)

# df_cust = df_cust.merge(
#     cust_agg,
#     on='customer_unique_id',
#     how='left'
# )

In [10]:
# filling the NaN with zeros 

df_cust['num_orders'] = df_cust['num_orders'].fillna(0)
df_cust['total_revenue'] = df_cust['total_revenue'].fillna(0)
df_cust['active'] = df_cust['active'].fillna(False)



In [13]:
df_ord.head(100)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delayed_days,order_completed,order_shipped,order_approved,order_total,total_freight,num_items,customer_unique_id
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,-8.0,True,True,True,29.99,8.72,1.0,7c396fd4830fd04220f754e42b4e5bff
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,-6.0,True,True,True,118.70,22.76,1.0,af07308b275d755c9edb36a90c618231
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,-18.0,True,True,True,159.90,19.22,1.0,3a653a41f6f9fc3d2a113cf8398680e8
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,-13.0,True,True,True,45.00,27.20,1.0,7c142cf63193a1473d2e66489a9ae977
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,-10.0,True,True,True,19.90,8.72,1.0,72632f0f9dd73dfee390c9b22eb56dd6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,77e9941864fc840be8e4b1ba5347c0f7,3135962ee745ef39b85576df7ddbaa99,delivered,2018-08-03 08:59:39,2018-08-03 09:31:36,2018-08-03 10:10:00,2018-08-17 00:49:41,2018-08-27,-10.0,True,True,True,65.90,37.37,1.0,00b2ca23369b68c4d4105ecea9c0cb93
96,41bb5cee06dbf170878a9ef93ac7e7f5,1833a0540067becaf59368fe4cd4303a,delivered,2018-05-14 08:35:33,2018-05-14 08:52:24,2018-05-16 14:46:00,2018-05-18 14:48:38,2018-06-08,-21.0,True,True,True,10.90,12.79,1.0,ca73adc05ad5d0d880de79b5ea3253b3
97,6a0a8bfbbe700284feb0845d95e0867f,68451b39b1314302c08c65a29f1140fc,delivered,2017-11-22 11:32:22,2017-11-22 11:46:50,2017-11-27 13:39:35,2017-12-28 19:43:00,2017-12-11,17.0,True,True,True,83.90,17.84,1.0,781ae350edb16842380e81d7c7feb431
98,f7959f8385f34c4f645327465a1c9fc4,0bf19317b1830a69e55b40710576aa7a,delivered,2017-03-30 07:50:33,2017-03-30 08:05:08,2017-03-30 10:55:54,2017-04-10 02:59:52,2017-04-26,-16.0,True,True,True,17.90,10.96,1.0,5ddb4fdd9cef2450d17ae20639815885


In [15]:
df_cust.to_csv('../data/cleaned/customers.csv', index=False)
df_ord.to_csv('../data/cleaned/orders.csv', index=False)